<a href="https://colab.research.google.com/github/webintellectual/ML_Summer_Internship/blob/main/05_Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files  

In [ ]:
files.upload() # (to upload kaggle.json)

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"webintellectual","key":"38199c4b8af327b53d4e92c6a1b62c11"}'}

In [ ]:
!mkdir ~/.kaggle
!mv /content/kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d slothkong/10-monkey-species
!unzip /content/10-monkey-species.zip 

100% 545M/547M [00:16<00:00, 41.1MB/s]
100% 547M/547M [00:16<00:00, 34.9MB/s]
Archive:  /content/10-monkey-species.zip
  inflating: monkey_labels.txt       
  inflating: training/training/n0/n0018.jpg  
  inflating: training/training/n0/n0019.jpg  
  inflating: training/training/n0/n0020.jpg  
  inflating: training/training/n0/n0021.jpg  
  inflating: training/training/n0/n0022.jpg  
  inflating: training/training/n0/n0023.jpg  
  inflating: training/training/n0/n0024.jpg  
  inflating: training/training/n0/n0025.jpg  
  inflating: training/training/n0/n0026.jpg  
  inflating: training/training/n0/n0027.jpg  
  inflating: training/training/n0/n0028.jpg  
  inflating: training/training/n0/n0029.jpg  
  inflating: training/training/n0/n0030.jpg  
  inflating: training/training/n0/n0031.jpg  
  inflating: training/training/n0/n0032.jpg  
  inflating: training/training/n0/n0033.jpg  
  inflating: training/training/n0/n0034.jpg  
  inflating: training/training/n0/n0035.jpg  
  inflating: tr

<hr>
Start of data preparation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(rescale=1/255.0)

In [ ]:
import os

In [ ]:
categories_list = os.listdir("training/training")

In [ ]:
total_training_images=0
total_validation_images=0
for folder_name in os.listdir("training/training"):
    total_training_images += len(os.listdir("training/training"+"/"+folder_name))
    total_validation_images+=len(os.listdir("validation/validation/"+folder_name))

print("There are {} images in training directory".format(total_training_images))
print("There are {} images in validation directory".format(total_validation_images))

There are 1098 images in training directory
There are 272 images in validation directory


In [ ]:
training_datagen = datagen.flow_from_directory(directory="training/training",
                                               target_size=(400,300),
                                               color_mode="rgb",
                                               classes=categories_list,
                                               batch_size=18)

Found 1098 images belonging to 10 classes.


In [ ]:
# If we use pre-trained weights then input size should also be equal to their defaut
training_datagen_224 = datagen.flow_from_directory(directory="training/training",
                                               target_size=(224,224),
                                               color_mode="rgb",
                                               classes=categories_list,
                                               batch_size=18)

Found 1098 images belonging to 10 classes.


In [ ]:
validation_datagen = datagen.flow_from_directory(directory="validation/validation",
                                               target_size=(400,300),
                                               color_mode="rgb",
                                               classes=categories_list,
                                               batch_size=272)

Found 272 images belonging to 10 classes.


In [ ]:
validation_datagen_224 = datagen.flow_from_directory(directory="validation/validation",
                                               target_size=(224,224),
                                               color_mode="rgb",
                                               classes=categories_list,
                                               batch_size=272)

Found 272 images belonging to 10 classes.


In [ ]:
def create_training_data_gen(n):
    training_datagen_n = datagen.flow_from_directory(directory="training/training",
                                               target_size=(n,n),
                                               color_mode="rgb",
                                               classes=categories_list,
                                               batch_size=18)
    return training_datagen

In [ ]:
def create_vald_data_gen(n):
    validation_datagen_n = datagen.flow_from_directory(directory="validation/validation",
                                               target_size=(n,n),
                                               color_mode="rgb",
                                               classes=categories_list,
                                               batch_size=272)
    return validation_datagen_n

End of data preparation
<hr>

In [ ]:
from keras.models import Model
from keras.layers import Dense

In [ ]:
from keras.layers import Flatten

In [ ]:
import tensorflow as tf

In [ ]:
def create_custom_cnn(pretrained_model):
    flattened_image = Flatten()(pretrained_model.output)
    fc1_output = Dense(units=512,activation="relu")(flattened_image)
    nw_output = Dense(units=10,activation="softmax")(fc1_output)
    
    our_custom_cnn = Model(inputs=pretrained_model.input, outputs=nw_output)
    return our_custom_cnn

In [ ]:
def replace_last_layer(pretrained_model):
    # Add a new dense layer with the desired number of units (number of classes)
    new_output = Dense(10, activation='softmax')(pretrained_model.layers[-2].output)
    # Connected the second last layer of this model to our new dense layer
    # Create a new model with the modified layers
    pretrained_with_wts_md = Model(inputs=pretrained_model.input, outputs=new_output)
    return pretrained_with_wts_md

In [ ]:
def freeze_pre_learned_weights(your_model):
    for layer in your_model.layers[:-1]:
        layer.trainable = False

<hr>

In [ ]:
from keras.applications import DenseNet121

DenseNet with pre-learned wts

In [ ]:
denseNet_with_wts = DenseNet121()

In [ ]:
denseNet_with_wts.layers
# This gives list of layers in the model

In [ ]:
denseNet_with_wts.summary()

Model: "densenet121"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 zero_padding2d_2 (ZeroPadd  (None, 230, 230, 3)          0         ['input_2[0][0]']             
 ing2D)                                                                                           
                                                                                                  
 conv1/conv (Conv2D)         (None, 112, 112, 64)         9408      ['zero_padding2d_2[0][0]']    
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1/conv[0][0]']

In [ ]:
denseNet_with_wts.layers.pop()
# This is of no use, It is just popping the last value from returned list of layers
# Does not affect the model architecture

We will use functional APIs

In [ ]:
# Add a new dense layer with the desired number of units (number of classes)
new_output = Dense(10, activation='softmax')(denseNet_with_wts.layers[-2].output)
# Connected the second last layer of this model to our new dense layer

In [ ]:
# Create a new model with the modified layers
denseNet_with_wts_md = Model(inputs=denseNet_with_wts.input, outputs=new_output)

In [ ]:
denseNet_with_wts_md.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 zero_padding2d_2 (ZeroPadd  (None, 230, 230, 3)          0         ['input_2[0][0]']             
 ing2D)                                                                                           
                                                                                                  
 conv1/conv (Conv2D)         (None, 112, 112, 64)         9408      ['zero_padding2d_2[0][0]']    
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1/conv[0][0]']    

In [ ]:
# Freeze the pre-trained layers 
for layer in denseNet_with_wts_md.layers[:-1]:
    layer.trainable = False

In [ ]:
denseNet_with_wts_md.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 zero_padding2d_2 (ZeroPadd  (None, 230, 230, 3)          0         ['input_2[0][0]']             
 ing2D)                                                                                           
                                                                                                  
 conv1/conv (Conv2D)         (None, 112, 112, 64)         9408      ['zero_padding2d_2[0][0]']    
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1/conv[0][0]']    

In [ ]:
denseNet_with_wts_md.compile(loss="categorical_crossentropy",
            metrics=["accuracy",tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

In [ ]:
denseNet_with_wts_md.fit(training_datagen_224,epochs=15,validation_data=validation_datagen_224)
# 13m 41.8s

Epoch 1/15


2023-06-03 11:43:56.352127: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


61/61 [==============================] - ETA: 0s - loss: 0.4699 - accuracy: 0.9126 - precision_1: 0.9764 - recall_1: 0.7914

2023-06-03 11:44:35.403462: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


61/61 [==============================] - 51s 803ms/step - loss: 0.4699 - accuracy: 0.9126 - precision_1: 0.9764 - recall_1: 0.7914 - val_loss: 0.3594 - val_accuracy: 0.9412 - val_precision_1: 0.9833 - val_recall_1: 0.8640
Epoch 2/15
61/61 [==============================] - 49s 799ms/step - loss: 0.2114 - accuracy: 0.9645 - precision_1: 0.9817 - recall_1: 0.9290 - val_loss: 0.2045 - val_accuracy: 0.9632 - val_precision_1: 0.9733 - val_recall_1: 0.9375
Epoch 3/15
61/61 [==============================] - 51s 836ms/step - loss: 0.1240 - accuracy: 0.9809 - precision_1: 0.9907 - recall_1: 0.9654 - val_loss: 0.1589 - val_accuracy: 0.9669 - val_precision_1: 0.9733 - val_recall_1: 0.9375
Epoch 4/15
61/61 [==============================] - 53s 866ms/step - loss: 0.0794 - accuracy: 0.9936 - precision_1: 0.9963 - recall_1: 0.9836 - val_loss: 0.1446 - val_accuracy: 0.9669 - val_precision_1: 0.9811 - val_recall_1: 0.9559
Epoch 5/15
61/61 [==============================] - 53s 872ms/step - loss: 0.05

<hr>

DenseNet with no pre-learned weights and including top

In [ ]:
denseNet_with_top = DenseNet121(weights=None,input_shape=(400,300,3),classes=10)

In [ ]:
denseNet_with_top.summary()

Model: "densenet121"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 400, 300, 3)]        0         []                            
                                                                                                  
 zero_padding2d_4 (ZeroPadd  (None, 406, 306, 3)          0         ['input_3[0][0]']             
 ing2D)                                                                                           
                                                                                                  
 conv1/conv (Conv2D)         (None, 200, 150, 64)         9408      ['zero_padding2d_4[0][0]']    
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 200, 150, 64)         256       ['conv1/conv[0][0]']

In [ ]:
denseNet_with_top.compile(loss="categorical_crossentropy",
            metrics=["accuracy",tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

In [ ]:
denseNet_with_top.fit(training_datagen,epochs=15,validation_data=validation_datagen)

Epoch 1/15


2023-06-03 12:07:15.834279: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


61/61 [==============================] - ETA: 0s - loss: 2.1868 - accuracy: 0.2404 - precision_2: 0.3956 - recall_2: 0.0656

2023-06-03 12:13:58.714421: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


61/61 [==============================] - 429s 7s/step - loss: 2.1868 - accuracy: 0.2404 - precision_2: 0.3956 - recall_2: 0.0656 - val_loss: 497.9031 - val_accuracy: 0.0993 - val_precision_2: 0.0993 - val_recall_2: 0.0993
Epoch 2/15
61/61 [==============================] - 441s 7s/step - loss: 1.8148 - accuracy: 0.3570 - precision_2: 0.6239 - recall_2: 0.1239 - val_loss: 4.1626 - val_accuracy: 0.0993 - val_precision_2: 0.0826 - val_recall_2: 0.0735
Epoch 3/15
61/61 [==============================] - 467s 8s/step - loss: 1.6703 - accuracy: 0.4162 - precision_2: 0.6292 - recall_2: 0.1530 - val_loss: 4.9779 - val_accuracy: 0.0772 - val_precision_2: 0.0549 - val_recall_2: 0.0184
Epoch 4/15
61/61 [==============================] - 508s 8s/step - loss: 1.5326 - accuracy: 0.4481 - precision_2: 0.6687 - recall_2: 0.1949 - val_loss: 4.3194 - val_accuracy: 0.1250 - val_precision_2: 0.2712 - val_recall_2: 0.0588
Epoch 5/15
61/61 [==============================] - 474s 8s/step - loss: 1.5176 - acc

<hr>

DenseNet with no pre-learned weights and not including top

In [ ]:
denseNet_without_top = DenseNet121(weights=None,input_shape=(400,300,3),include_top=False)

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-06-03 14:26:07.467695: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-06-03 14:26:07.467952: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
denseNet_without_top.summary()

Model: "densenet121"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 400, 300, 3)]        0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 406, 306, 3)          0         ['input_1[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv1/conv (Conv2D)         (None, 200, 150, 64)         9408      ['zero_padding2d[0][0]']      
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 200, 150, 64)         256       ['conv1/conv[0][0]']

<hr>

In [ ]:
denseNet_without_top_md = create_custom_cnn(denseNet_without_top)

In [ ]:
denseNet_without_top_md.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 400, 300, 3)]        0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 406, 306, 3)          0         ['input_1[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv1/conv (Conv2D)         (None, 200, 150, 64)         9408      ['zero_padding2d[0][0]']      
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 200, 150, 64)         256       ['conv1/conv[0][0]']      

In [ ]:
denseNet_without_top_md.compile(loss="categorical_crossentropy",
            metrics=["accuracy",tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

In [ ]:
denseNet_without_top_md.fit(training_datagen,epochs=15,validation_data=validation_datagen)

Epoch 1/15


2023-06-03 15:02:13.293392: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


61/61 [==============================] - 141s 2s/step - loss: 1.7458 - accuracy: 0.5155 - precision: 0.6178 - recall: 0.3725 - val_loss: 137.2661 - val_accuracy: 0.1287 - val_precision: 0.1261 - val_recall: 0.1029
Epoch 2/15
61/61 [==============================] - 153s 3s/step - loss: 1.1512 - accuracy: 0.6138 - precision: 0.6974 - recall: 0.5036 - val_loss: 4.5894 - val_accuracy: 0.3162 - val_precision: 0.3713 - val_recall: 0.2279
Epoch 3/15
61/61 [==============================] - 161s 3s/step - loss: 0.9562 - accuracy: 0.7022 - precision: 0.7847 - recall: 0.5974 - val_loss: 3026.9250 - val_accuracy: 0.1434 - val_precision: 0.1434 - val_recall: 0.1434
Epoch 4/15
61/61 [==============================] - 169s 3s/step - loss: 0.8085 - accuracy: 0.7450 - precision: 0.8149 - recall: 0.6658 - val_loss: 1255.4977 - val_accuracy: 0.1618 - val_precision: 0.1533 - val_recall: 0.1471
Epoch 5/15
61/61 [==============================] - 160s 3s/step - loss: 0.5867 - accuracy: 0.7996 - precision:

<hr>

**Inception**

In [ ]:
from keras.applications import InceptionV3

With pre-learned wts

In [ ]:
inception_with_wts = InceptionV3()

96112376/96112376 [==============================] - 67s 1us/step


In [ ]:
inception_with_wts.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 149, 149, 32)         864       ['input_3[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 149, 149, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 149, 149, 32)         0         ['batch_normalizati

In [ ]:
inception_with_wts_md = replace_last_layer(inception_with_wts)

In [ ]:
inception_with_wts_md.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 149, 149, 32)         864       ['input_3[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 149, 149, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 149, 149, 32)         0         ['batch_normalization[0]

In [ ]:
freeze_pre_learned_weights(inception_with_wts_md)

In [ ]:
inception_with_wts_md.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 149, 149, 32)         864       ['input_3[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 149, 149, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 149, 149, 32)         0         ['batch_normalization[0]

In [ ]:
training_datagen_299 = create_training_data_gen(299)

Found 1098 images belonging to 10 classes.


In [ ]:
validation_datagen_299 = create_vald_data_gen(299)

Found 272 images belonging to 10 classes.


In [ ]:
inception_with_wts_md.compile(loss="categorical_crossentropy",
            metrics=["accuracy",tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

In [ ]:
inception_with_wts_md.fit(training_datagen_299,epochs=15,validation_data=validation_datagen_299)

Epoch 1/15


2023-06-03 16:10:52.036031: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-03 16:10:53.274926: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


61/61 [==============================] - ETA: 0s - loss: 0.8323 - accuracy: 0.8616 - precision_2: 0.9932 - recall_2: 0.5355

2023-06-03 16:11:18.564791: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-03 16:11:18.879311: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


61/61 [==============================] - 35s 499ms/step - loss: 0.8323 - accuracy: 0.8616 - precision_2: 0.9932 - recall_2: 0.5355 - val_loss: 0.1521 - val_accuracy: 0.9890 - val_precision_2: 0.9888 - val_recall_2: 0.9706
Epoch 2/15
61/61 [==============================] - 24s 392ms/step - loss: 0.1694 - accuracy: 0.9809 - precision_2: 0.9896 - recall_2: 0.9536 - val_loss: 0.0695 - val_accuracy: 0.9890 - val_precision_2: 0.9889 - val_recall_2: 0.9853
Epoch 3/15
61/61 [==============================] - 24s 401ms/step - loss: 0.0833 - accuracy: 0.9872 - precision_2: 0.9907 - recall_2: 0.9754 - val_loss: 0.0373 - val_accuracy: 0.9890 - val_precision_2: 0.9890 - val_recall_2: 0.9890
Epoch 4/15
61/61 [==============================] - 28s 445ms/step - loss: 0.0476 - accuracy: 0.9945 - precision_2: 0.9963 - recall_2: 0.9918 - val_loss: 0.0387 - val_accuracy: 0.9890 - val_precision_2: 0.9926 - val_recall_2: 0.9890
Epoch 5/15
61/61 [==============================] - 30s 491ms/step - loss: 0.03

Inception without weights and with top

In [ ]:
inception_with_top = InceptionV3(weights=None,input_shape=(400,300,3),classes=10)

In [ ]:
inception_with_top.compile(loss="categorical_crossentropy",
            metrics=["accuracy",tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

In [ ]:
inception_with_top.fit(training_datagen,epochs=15,validation_data=validation_datagen)

Epoch 1/15
61/61 [==============================] - 119s 1s/step - loss: 2.5144 - accuracy: 0.1985 - precision: 0.2119 - recall: 0.0291 - val_loss: 3.5161 - val_accuracy: 0.1103 - val_precision: 0.1107 - val_recall: 0.0993
Epoch 2/15
61/61 [==============================] - 44s 721ms/step - loss: 2.1881 - accuracy: 0.2204 - precision: 0.3827 - recall: 0.0565 - val_loss: 276.7219 - val_accuracy: 0.0993 - val_precision: 0.0993 - val_recall: 0.0993
Epoch 3/15
61/61 [==============================] - 44s 722ms/step - loss: 2.0216 - accuracy: 0.2322 - precision: 0.4750 - recall: 0.0519 - val_loss: 2.8205 - val_accuracy: 0.0919 - val_precision: 0.0488 - val_recall: 0.0074
Epoch 4/15
61/61 [==============================] - 45s 730ms/step - loss: 1.7518 - accuracy: 0.3634 - precision: 0.6079 - recall: 0.1257 - val_loss: 3.3096 - val_accuracy: 0.1507 - val_precision: 0.2710 - val_recall: 0.1066
Epoch 5/15
61/61 [==============================] - 44s 713ms/step - loss: 1.6482 - accuracy: 0.3852

Inception without weights and without top

In [ ]:
inception_without_top = InceptionV3(include_top=False,weights=None,input_shape=(400,300,3))

In [ ]:
inception_without_top_md = create_custom_cnn(inception_without_top)

In [ ]:
inception_without_top_md.compile(loss="categorical_crossentropy",
            metrics=["accuracy",tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

In [ ]:
inception_without_top_md.fit(training_datagen,epochs=15,validation_data=validation_datagen)

Epoch 1/15
61/61 [==============================] - 82s 910ms/step - loss: 31.0795 - accuracy: 0.0965 - precision_1: 0.0917 - recall_1: 0.0546 - val_loss: 2.3415 - val_accuracy: 0.1029 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 2/15
61/61 [==============================] - 44s 711ms/step - loss: 2.5842 - accuracy: 0.1384 - precision_1: 0.1481 - recall_1: 0.0073 - val_loss: 2.3122 - val_accuracy: 0.0993 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 3/15
61/61 [==============================] - 44s 725ms/step - loss: 2.2965 - accuracy: 0.1084 - precision_1: 0.2500 - recall_1: 0.0018 - val_loss: 65.0565 - val_accuracy: 0.0919 - val_precision_1: 0.0919 - val_recall_1: 0.0919
Epoch 4/15
61/61 [==============================] - 45s 727ms/step - loss: 2.3159 - accuracy: 0.1157 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00 - val_loss: 2.9037 - val_accuracy: 0.1103 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 5/15
61/61 [================

<hr>

**ResNet50V2**

In [ ]:
from keras.applications import ResNet50V2

With weights

In [ ]:
resnet_with_wts = ResNet50V2()

102869336/102869336 [==============================] - 4s 0us/step


In [ ]:
resnet_with_wts_md = replace_last_layer(resnet_with_wts)

In [ ]:
freeze_pre_learned_weights(resnet_with_wts_md)

In [ ]:
resnet_with_wts_md.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_3[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [ ]:
training_datagen_224 = create_training_data_gen(224)

Found 1098 images belonging to 10 classes.


In [ ]:
validation_datagen_224 = create_vald_data_gen(224)

Found 272 images belonging to 10 classes.


In [ ]:
resnet_with_wts_md.compile(loss="categorical_crossentropy",
            metrics=["accuracy",tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

In [ ]:
resnet_with_wts_md.fit(training_datagen_224,epochs=15,validation_data=validation_datagen_224)

Epoch 1/15
61/61 [==============================] - 65s 991ms/step - loss: 0.8142 - accuracy: 0.8552 - precision_2: 0.9874 - recall_2: 0.5692 - val_loss: 0.1722 - val_accuracy: 0.9449 - val_precision_2: 0.9658 - val_recall_2: 0.9338
Epoch 2/15
61/61 [==============================] - 42s 684ms/step - loss: 0.1412 - accuracy: 0.9891 - precision_2: 0.9935 - recall_2: 0.9672 - val_loss: 0.1002 - val_accuracy: 0.9522 - val_precision_2: 0.9625 - val_recall_2: 0.9449
Epoch 3/15
61/61 [==============================] - 42s 700ms/step - loss: 0.0538 - accuracy: 0.9964 - precision_2: 0.9972 - recall_2: 0.9891 - val_loss: 0.0986 - val_accuracy: 0.9596 - val_precision_2: 0.9665 - val_recall_2: 0.9559
Epoch 4/15
61/61 [==============================] - 41s 679ms/step - loss: 0.0254 - accuracy: 0.9991 - precision_2: 0.9991 - recall_2: 0.9964 - val_loss: 0.1106 - val_accuracy: 0.9632 - val_precision_2: 0.9667 - val_recall_2: 0.9596
Epoch 5/15
61/61 [==============================] - 42s 688ms/step -

Without weights and with top

In [ ]:
resnet_with_top = ResNet50V2(weights=None,input_shape=(400,300,3),classes=10)

In [ ]:
resnet_with_top.compile(loss="categorical_crossentropy",
            metrics=["accuracy",tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

In [ ]:
resnet_with_top.fit(training_datagen,epochs=15,validation_data=validation_datagen)

Epoch 1/15
61/61 [==============================] - 104s 1s/step - loss: 2.3170 - accuracy: 0.2259 - precision_3: 0.4231 - recall_3: 0.0601 - val_loss: 27.1559 - val_accuracy: 0.0956 - val_precision_3: 0.0963 - val_recall_3: 0.0956
Epoch 2/15
61/61 [==============================] - 48s 781ms/step - loss: 1.9692 - accuracy: 0.2896 - precision_3: 0.5513 - recall_3: 0.0783 - val_loss: 4.4924 - val_accuracy: 0.2022 - val_precision_3: 0.2518 - val_recall_3: 0.1287
Epoch 3/15
61/61 [==============================] - 47s 781ms/step - loss: 1.7821 - accuracy: 0.3588 - precision_3: 0.6306 - recall_3: 0.1275 - val_loss: 2.7836 - val_accuracy: 0.2426 - val_precision_3: 0.2688 - val_recall_3: 0.0919
Epoch 4/15
61/61 [==============================] - 48s 763ms/step - loss: 1.6308 - accuracy: 0.4089 - precision_3: 0.6755 - recall_3: 0.1630 - val_loss: 7.6226 - val_accuracy: 0.0993 - val_precision_3: 0.1004 - val_recall_3: 0.0956
Epoch 5/15
61/61 [==============================] - 47s 776ms/step - 

Without weights and without top

In [ ]:
resnet_without_top = ResNet50V2(include_top=False,weights=None,input_shape=(400,300,3))

In [ ]:
resnet_without_top_md = create_custom_cnn(resnet_without_top)

In [ ]:
resnet_without_top_md.compile(loss="categorical_crossentropy",
            metrics=["accuracy",tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

In [ ]:
resnet_without_top_md.fit(training_datagen,epochs=15,validation_data=validation_datagen)

Epoch 1/15
61/61 [==============================] - 85s 996ms/step - loss: 49.1833 - accuracy: 0.1621 - precision_4: 0.1590 - recall_4: 0.1202 - val_loss: 3.4279 - val_accuracy: 0.0919 - val_precision_4: 0.1111 - val_recall_4: 0.0221
Epoch 2/15
61/61 [==============================] - 51s 824ms/step - loss: 3.5034 - accuracy: 0.2013 - precision_4: 0.2450 - recall_4: 0.0556 - val_loss: 2.3705 - val_accuracy: 0.1250 - val_precision_4: 0.0000e+00 - val_recall_4: 0.0000e+00
Epoch 3/15
61/61 [==============================] - 49s 808ms/step - loss: 2.0096 - accuracy: 0.3424 - precision_4: 0.4739 - recall_4: 0.1157 - val_loss: 2.8271 - val_accuracy: 0.1471 - val_precision_4: 0.3158 - val_recall_4: 0.0882
Epoch 4/15
61/61 [==============================] - 50s 811ms/step - loss: 1.9964 - accuracy: 0.4171 - precision_4: 0.5502 - recall_4: 0.2195 - val_loss: 9.3166 - val_accuracy: 0.0919 - val_precision_4: 0.0947 - val_recall_4: 0.0662
Epoch 5/15
61/61 [==============================] - 49s 794

<hr>

**VGG16**

In [ ]:
from keras.applications import VGG16

With weights

In [ ]:
vgg16_with_wts = VGG16()

In [ ]:
vgg16_with_wts_md = replace_last_layer(vgg16_with_wts)

In [ ]:
vgg16_with_wts_md.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
training_datagen_224 = create_training_data_gen(224)

Found 1098 images belonging to 10 classes.


In [ ]:
validation_datagen = create_vald_data_gen(224)

Found 272 images belonging to 10 classes.


In [ ]:
vgg16_with_wts_md.compile(loss="categorical_crossentropy",
            metrics=["accuracy",tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

In [ ]:
vgg16_with_wts_md.fit(training_datagen_224,epochs=15,validation_data=validation_datagen_224)

Epoch 1/15


InvalidArgumentError: ignored

Without weights and with top

In [ ]:
vgg16_with_top = VGG16(weights=None,input_shape=(400,300,3),classes=10)

In [ ]:
vgg16_with_top.compile(loss="categorical_crossentropy",
            metrics=["accuracy",tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

In [ ]:
vgg16_with_top.fit(training_datagen,epochs=15,validation_data=validation_datagen)

Without weights and without top

In [ ]:
vgg16_without_top = VGG16(include_top=False,weights=None,input_shape=(400,300,3))

In [ ]:
vgg16_without_top_md = create_custom_cnn(vgg16_without_top)